## Load documents with IDs

In [1]:
import json

with open('/app/Law_corpus/documents-with-ids.json', 'rt') as f_in:  # Use /app, not the Windows path
    documents = json.load(f_in)

In [2]:
documents[10]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 11',
 'title': 'Ngôn ngữ, chữ viết dùng trong cơ sở giáo dục',
 'content': '1. Tiếng Việt là ngôn ngữ chính thức dùng trong cơ sở giáo dục. Căn cứ vào mục tiêu giáo dục và yêu cầu cụ thể về nội dung giáo dục, Chính phủ quy định việc dạy và học bằng tiếng nước ngoài trong cơ sở giáo dục. 2. Nhà nước khuyến khích, tạo điều kiện để người dân tộc thiểu số được học tiếng nói, chữ viết của dân tộc mình theo quy định của Chính phủ; người khuyết tật nghe, nói được học bằng ngôn ngữ ký hiệu, người khuyết tật nhìn được học bằng chữ nổi Braille theo quy định của Luật Người khuyết tật. 3. Ngoại ngữ quy định trong chương trình giáo dục là ngôn ngữ được sử dụng phổ biến trong giao dịch quốc tế. Việc tổ chức dạy ngoại ngữ trong cơ sở giáo dục phải bảo đảm để người học được học liên tục, hiệu quả. ',
 'id': '1095c99a'}

## Load ground truth

In [3]:
import pandas as pd

df_ground_truth = pd.read_csv('./evaluate/ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [4]:
ground_truth[10]

{'questions': 'Nền giáo dục Việt Nam có đặc điểm gì nổi bật?',
 'law_title': 'Luật Giáo Dục',
 'document': 'e0e71ffa'}

In [5]:
doc_idx = {d['id']: d for d in documents}
doc_idx['e0e71ffa']

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 3',
 'title': 'Tính chất, nguyên lý giáo dục',
 'content': '1. Nền giáo dục Việt Nam là nền giáo dục xã hội chủ nghĩa có tính nhân dân, dân tộc, khoa học, hiện đại, lấy chủ nghĩa Mác - Lê nin và tư tưởng Hồ Chí Minh làm nền tảng. 2. Hoạt động giáo dục được thực hiện theo nguyên lý học đi đôi với hành, lý luận gắn liền với thực tiễn, giáo dục nhà trường kết hợp với giáo dục gia đình và giáo dục xã hội. ',
 'id': 'e0e71ffa'}

## Index data

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("truro7/vn-law-embedding", truncate_dim = 768) #truncate_dim = 768

Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.


In [7]:
from elasticsearch import Elasticsearch
# es_client = Elasticsearch('http://localhost:9200') 
es_client = Elasticsearch('http://elasticsearch_compose:9200') #connect to docker compose network if runinng from dockerfile container 
# es_client.info()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "law_title": {"type": "keyword"},                
            "law_number": {"type": "keyword"},               
            "chapter_title": {"type": "keyword"},            
            "article_number": {"type": "keyword"},            
            "title": {"type": "text"},                        
            "content": {"type": "text"},   
            "id": {"type": "text"},   
            "title_content_vector": {                                 
                "type": "dense_vector",
                "dims": 768,                                 
                "index": True,
                "similarity": "cosine"
            }                
        }
    }
}

index_name = "legal_documents_ids"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'legal_documents_ids'})

In [8]:
documents[10]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 11',
 'title': 'Ngôn ngữ, chữ viết dùng trong cơ sở giáo dục',
 'content': '1. Tiếng Việt là ngôn ngữ chính thức dùng trong cơ sở giáo dục. Căn cứ vào mục tiêu giáo dục và yêu cầu cụ thể về nội dung giáo dục, Chính phủ quy định việc dạy và học bằng tiếng nước ngoài trong cơ sở giáo dục. 2. Nhà nước khuyến khích, tạo điều kiện để người dân tộc thiểu số được học tiếng nói, chữ viết của dân tộc mình theo quy định của Chính phủ; người khuyết tật nghe, nói được học bằng ngôn ngữ ký hiệu, người khuyết tật nhìn được học bằng chữ nổi Braille theo quy định của Luật Người khuyết tật. 3. Ngoại ngữ quy định trong chương trình giáo dục là ngôn ngữ được sử dụng phổ biến trong giao dịch quốc tế. Việc tổ chức dạy ngoại ngữ trong cơ sở giáo dục phải bảo đảm để người học được học liên tục, hiệu quả. ',
 'id': '1095c99a'}

In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    title = doc['title']
    content = doc['content']
    doc['title_content_vector'] = model.encode(title + ' ' + content)

    es_client.index(index=index_name, document=doc)

  0%|          | 0/314 [00:00<?, ?it/s]

## Retrieval

In [58]:
def elastic_search_knn(field, vector, law_title):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 1,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "law_title": law_title
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["law_title", "law_number", "chapter_title", "article_number", "title", "content","id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def title_content_vector_knn(q):
    questions = q['questions']
    law_title = q['law_title']

    v_q = model.encode(questions)

    return elastic_search_knn('title_content_vector', v_q, law_title)


In [67]:
title_content_vector_knn(ground_truth[1])

[{'article_number': 'Điều 82',
  'law_number': 'Luật số: 43/2019/QH14',
  'chapter_title': 'Chương V',
  'id': '2fa13653',
  'title': 'Nhiệm vụ của người học',
  'law_title': 'Luật Giáo Dục',
  'content': '1. Học tập, rèn luyện theo chương trình, kế hoạch giáo dục, quy tắc ứng xử của cơ sở giáo dục. 2. Tôn trọng nhà giáo, cán bộ và người lao động của cơ sở giáo dục; đoàn kết, giúp đỡ lẫn nhau trong học tập, rèn luyện; thực hiện nội quy, điều lệ, quy chế của cơ sở giáo dục; chấp hành quy định của pháp luật. 3. Tham gia lao động và hoạt động xã hội, hoạt động bảo vệ môi trường phù hợp với lứa tuổi, sức khỏe và năng lực. 4. Giữ gìn, bảo vệ tài sản của cơ sở giáo dục. 5. Góp phần xây dựng, bảo vệ và phát huy truyền thống của cơ sở giáo dục. '}]

## The RAG flow

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a law searcher assistant. Answer the QUESTION based on the CONTEXT from the law database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"law_title: {doc['law_title']}\nlaw_number: {doc['law_number']}\narticle_number: {doc['article_number']}\ntitle: {doc['title']}\ncontent: {doc['content']}\n\n"

    # prompt_template = """CONTEXT:\n{context}\n\nQUESTION: {question}""".strip()
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



In [ ]:
from googletrans import Translator

# Initialize the translator
translator = Translator()

def vietnamese_to_english(text):
    """
    Convert Vietnamese text to English.
    
    :param text: Vietnamese text to be translated
    :return: Translated English text
    """
    try:
        translation = translator.translate(text, src='vi', dest='en')
        return translation.text
    except Exception as e:
        return f"An error occurred: {e}"

def english_to_vietnamese(text):
    """
    Convert English text to Vietnamese.
    
    :param text: English text to be translated
    :return: Translated Vietnamese text
    """
    try:
        translation = translator.translate(text, src='en', dest='vi')
        return translation.text
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
if __name__ == "__main__":
    vietnamese_text = "Luật việt nam có rất nhiều điều khoản.và thuộc nhiều bộ luật khác nhau"
    print(f"Original Vietnamese: {vietnamese_text}")

    
    english_text = vietnamese_to_english(vietnamese_text)
    print(f"Vietnamese to English: {english_text}")

    converted_back_text = english_to_vietnamese(english_text)
    print(f"English back to Vietnamese: {converted_back_text}")
context = build_prompt(ground_truth[1], title_content_vector_knn(ground_truth[1]))
context


In [76]:
# update 29-12
from googletrans import Translator
def clean_text(text):
    """
    Cleans text by removing non-printable characters and normalizing whitespace.
    """
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single spaces
    text = ''.join(c for c in text if c.isprintable())  # Remove non-printable characters
    return text

def build_prompt(query, search_results):
    prompt_template = """
Sei un assistente di ricerca legale. Rispondi alla DOMANDA basandoti sul CONTESTO del database legale.
Utilizza solo i fatti dal CONTESTO quando rispondi alla DOMANDA.

DOMANDA: {query}

CONTESTO:
{context}
""".strip()

    # Translate the query to Italian
    translated_query = vietnamese_to_italian(query)

    context = ""
    for doc in search_results:
        # Translate each field of the document to Italian
        translated_law_title = vietnamese_to_italian(clean_text(doc['law_title']))
        translated_law_number = vietnamese_to_italian(clean_text(doc['law_number']))
        translated_article_number = vietnamese_to_italian(clean_text(doc['article_number']))
        translated_title = vietnamese_to_italian(clean_text(doc['title']))
        translated_content = vietnamese_to_italian(clean_text(doc['content']))
        
        context += f"titolo_legge: {translated_law_title}\nnumero_legge: {translated_law_number}\nnumero_articolo: {translated_article_number}\ntitolo: {translated_title}\ncontenuto: {translated_content}\n\n"

    prompt = prompt_template.format(question=translated_query, context=context).strip()
    return prompt

translator = Translator()

def vietnamese_to_italian(text):
    """
    Translates Vietnamese text to Italian using the googletrans library.
    Handles potential None values returned by the translator.
    """
    try:
        if text is None:  # Check for None input
            return ""
        translation = translator.translate(text, src='vi', dest='it')
        if translation is None: # Check for None translation result
            return ""
        return translation.text
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        return ""
def italian_to_vietnamese(text):
    """
    Translates Italian text to Vietnamese using the googletrans library.
    Handles potential None values returned by the translator.
    """
    try:
        if text is None:  # Check for None input
            return ""
        translation = translator.translate(text, src='it', dest='vi')
        if translation is None:  # Check for None translation result
            return ""
        return translation.text
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        return ""

In [75]:
context = build_prompt(ground_truth[0], title_content_vector_knn(ground_truth[0]))
context


An error occurred during translation: the JSON object must be str, bytes or bytearray, not NoneType


"Sei un assistente di ricerca legale. Rispondi alla DOMANDA basandoti sul CONTESTO del database legale.\nUtilizza solo i fatti dal CONTESTO quando rispondi alla DOMANDA.\n\nDOMANDA: \n\nCONTESTO:\ntitolo_legge: Legge sull'istruzione\nnumero_legge: Legge n. 43/2019/QH14\nnumero_articolo: Articolo 1\ntitolo: Ambito di regolazione\ncontenuto: Questa legge stabilisce il sistema educativo nazionale;istituzioni educative, insegnanti, studenti;Gestione statale dell'educazione;Diritti e responsabilità di agenzie, organizzazioni e individui relativi alle attività educative."

In [78]:
title_content_vector_knn(ground_truth[0])

[{'article_number': 'Điều 1',
  'law_number': 'Luật số: 43/2019/QH14',
  'chapter_title': 'Chương I',
  'id': '75a9286e',
  'title': 'Phạm vi điều chỉnh',
  'law_title': 'Luật Giáo Dục',
  'content': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. '}]

In [77]:
print(italian_to_vietnamese(context))

Bạn là một trợ lý nghiên cứu pháp lý.Trả lời câu hỏi dựa trên bối cảnh của cơ sở dữ liệu pháp lý.
Chỉ sử dụng các sự kiện từ bối cảnh khi bạn trả lời câu hỏi.

LỜI YÊU CẦU:

BỐI CẢNH:
Tiêu đề_legge: Luật giáo dục
Number_legge: Luật số.43/2019/QH14
Number_ Bài báo: Điều 1
Tiêu đề: Khu vực quy định
Nội dung: Luật này thiết lập hệ thống giáo dục quốc gia;


In [13]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import torch

mamba_model_name = "DeepMount00/mamba_790_hf_qa"
mamba_tokenizer = AutoTokenizer.from_pretrained(mamba_model_name)
mamba_model = MambaForCausalLM.from_pretrained(mamba_model_name, device_map={"": 0}).eval()

The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the sequential implementation of Mamba, as use_mambapy is set to False. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. For the mamba.py backend, follow https://github.com/alxndrTL/mamba.py.


In [63]:
def llm(context, question):
    device = "cuda:0"
    # Hybrid prefix: Start with the working Italian, then transition to English instructions

    prompt = f"""##CONTESTO: {context}\n##DOMANDA: {question}\n"""

    input_ids = mamba_tokenizer([prompt], return_tensors="pt").to(device)

    generate_ids = mamba_model.generate(**input_ids, max_new_tokens=300, eos_token_id=8112)

    answer = mamba_tokenizer.batch_decode(generate_ids)

    try:
        final_answer = answer[0].split("##RISPOSTA: ")[1].split("##END")[0].strip("\n")
    except IndexError:
        final_answer = ""
    return final_answer

In [64]:
context = """La torre degli Asinelli è una delle cosiddette due torri di Bologna, simbolo della città, situate in piazza di porta Ravegnana, all'incrocio tra le antiche strade San Donato (ora via Zamboni), San Vitale, Maggiore e Castiglione. Eretta, secondo la tradizione, fra il 1109 e il 1119 dal nobile Gherardo Asinelli, la torre è alta 97,20 metri, pende verso ovest per 2,23 metri e presenta all'interno una scalinata composta da 498 gradini. Ancora non si può dire con certezza quando e da chi fu costruita la torre degli Asinelli. Si presume che la torre debba il proprio nome a Gherardo Asinelli, il nobile cavaliere di fazione ghibellina al quale se ne attribuisce la costruzione, iniziata secondo una consolidata tradizione l'11 ottobre 1109 e terminata dieci anni dopo, nel 1119."""
question = "Dove si trova precisamente la torre degli Asinelli?"
print(llm(context, question))

La torre degli Asinelli si trova in piazza di porta Ravegnana, all'incrocio tra le antiche strade San Donato (ora via Zamboni, San Vitale, Maggiore e Castiglione).


In [17]:
from typing import List, Dict

def vietnamese_to_english(search_results: List[Dict]) -> List[Dict]:
    """
    Translates the Vietnamese text in search results to English.

    Args:
        search_results: A list of dictionaries, where each dictionary represents a search result 
                        with Vietnamese text in keys like 'law_title', 'title', and 'content'.

    Returns:
        A list of dictionaries with the translated English text.
    """
    translated_results = []
    for doc in search_results:
        translated_doc = {
            "law_title": translator.translate(doc["law_title"], src="vi", dest="en").text,
            "law_number": doc["law_number"],  # Assuming these don't need translation
            "article_number": doc["article_number"],
            "title": translator.translate(doc["title"], src="vi", dest="en").text,
            "content": translator.translate(doc["content"], src="vi", dest="en").text,
        }
        translated_results.append(translated_doc)
    return translated_results

def english_to_vietnamese(text: str) -> str:
    """
    Translates English text to Vietnamese.

    Args:
        text: The English text to translate.

    Returns:
        The translated Vietnamese text.
    """
    try:
        translation = translator.translate(text, src="en", dest="vi")
        return translation.text
    except Exception as e:
        return f"An error occurred: {e}"

In [60]:
title_content_vector_knn(ground_truth[1])

[{'article_number': 'Điều 82',
  'law_number': 'Luật số: 43/2019/QH14',
  'chapter_title': 'Chương V',
  'id': '2fa13653',
  'title': 'Nhiệm vụ của người học',
  'law_title': 'Luật Giáo Dục',
  'content': '1. Học tập, rèn luyện theo chương trình, kế hoạch giáo dục, quy tắc ứng xử của cơ sở giáo dục. 2. Tôn trọng nhà giáo, cán bộ và người lao động của cơ sở giáo dục; đoàn kết, giúp đỡ lẫn nhau trong học tập, rèn luyện; thực hiện nội quy, điều lệ, quy chế của cơ sở giáo dục; chấp hành quy định của pháp luật. 3. Tham gia lao động và hoạt động xã hội, hoạt động bảo vệ môi trường phù hợp với lứa tuổi, sức khỏe và năng lực. 4. Giữ gìn, bảo vệ tài sản của cơ sở giáo dục. 5. Góp phần xây dựng, bảo vệ và phát huy truyền thống của cơ sở giáo dục. '}]

In [34]:
def rag(query: str) -> str:
    search_results = title_content_vector_knn(query)
    #1 vi to en
    en_search_results = vietnamese_to_english(search_results)
    context = build_prompt(query, en_search_results)  # Context is built from search results
    answer = llm(context, query)  # Pass context and query to llm
    #2 en to vi
    vi_answer = english_to_vietnamese(answer)
    return vi_answer

In [22]:
ground_truth[1]

{'questions': 'Những quyền và trách nhiệm nào của cá nhân liên quan đến hoạt động giáo dục được quy định trong Luật này?',
 'law_title': 'Luật Giáo Dục',
 'document': '75a9286e'}

In [37]:
rag(ground_truth[1])

'Quên tên của đất nước cụ thể trong luật.Thông báo nó thông qua khảo sát.'

In [36]:
doc_idx['75a9286e']['content']

'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. '

## Cosine similarity metric

In [39]:
law_content = 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. '

answer_llm = "'Có những blog khác thảo luận về chủ đề' Có an toàn không khi sử dụng nhân sâm ''?"


v_llm = model.encode(answer_llm)
v_orig = model.encode(law_content)

v_llm.dot(v_orig)

np.float32(-0.05068712)

In [28]:
ground_truth[0]

{'questions': 'Luật Giáo Dục điều chỉnh những đối tượng nào trong hệ thống giáo dục quốc dân?',
 'law_title': 'Luật Giáo Dục',
 'document': '75a9286e'}

In [29]:
len(ground_truth)

1570

In [30]:
answers = {}

In [44]:
for i, rec in enumerate(tqdm(ground_truth)):
    if i in answers:
        continue

    answer_llm = rag(rec)
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    law_content = original_doc['content']

    answers[i] = {
        'answer_llm': answer_llm,
        'answer_orig': law_content,
        'document': doc_id,
        'questions': rec['questions'],tq
        'law_title': rec['law_title'],
    }

  0%|          | 0/1570 [00:00<?, ?it/s]

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [47]:
len(answers)

535

In [53]:
answers.values()

dict_values([{'answer_llm': 'quan ho trang', 'answer_orig': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ', 'document': '75a9286e', 'questions': 'Luật Giáo Dục điều chỉnh những đối tượng nào trong hệ thống giáo dục quốc dân?', 'law_title': 'Luật Giáo Dục'}, {'answer_llm': 'Cosa si deve fare Quando un figlio si trova trong prefettura?', 'answer_orig': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ', 'document': '75a9286e', 'questions': 'Những quyền và trách nhiệm nào của cá nhân liên quan đến hoạt động giáo dục được quy định trong Luật này?', 'law_title': 'Luật Giáo Dục'}, {'answer_llm': 'Giái', 'answer_orig': 'Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà 

In [57]:
df = pd.DataFrame(answers.values())
df.to_csv("mamba_en_vi.csv", index=False)


In [ ]:
results_gpt4o = [None] * len(ground_truth)

for i, val in answers.items():
    results_gpt4o[i] = val.copy()
    results_gpt4o[i].update(ground_truth[i])

In [54]:
import pandas as pd

In [ ]:
df_gpt4o = pd.DataFrame(results_gpt4o)

In [ ]:
!mkdir data

In [ ]:
df_gpt4o.to_csv('data/results-gpt4o.csv', index=False)

## Evaluating GPT 3.5

In [ ]:
rag(ground_truth[10], model='gpt-3.5-turbo')

In [ ]:
from tqdm.auto import tqdm

from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [ ]:
def process_record(rec):
    model = 'gpt-3.5-turbo'
    answer_llm = rag(rec, model=model)
    
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['text']

    return {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course'],
    }

In [ ]:
process_record(ground_truth[10])

In [ ]:
results_gpt35 = map_progress(pool, ground_truth, process_record)

In [ ]:
df_gpt35 = pd.DataFrame(results_gpt35)
df_gpt35.to_csv('data/results-gpt35.csv', index=False)

In [ ]:
!head data/results-gpt35.csv

## Cosine similarity

A->Q->A' cosine similarity

A -> Q -> A'

cosine(A, A')

### gpt-4o

In [ ]:
results_gpt4o = df_gpt4o.to_dict(orient='records')

In [ ]:
record = results_gpt4o[0]

In [ ]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [ ]:
similarity = []

for record in tqdm(results_gpt4o):
    sim = compute_similarity(record)
    similarity.append(sim)

In [ ]:
df_gpt4o['cosine'] = similarity
df_gpt4o['cosine'].describe()

In [ ]:
import seaborn as sns

### gpt-3.5-turbo

In [ ]:
results_gpt35 = df_gpt35.to_dict(orient='records')

similarity_35 = []

for record in tqdm(results_gpt35):
    sim = compute_similarity(record)
    similarity_35.append(sim)

In [ ]:
df_gpt35['cosine'] = similarity_35
df_gpt35['cosine'].describe()

In [ ]:
import matplotlib.pyplot as plt

### gpt-4o-mini

In [ ]:
def process_record_4o_mini(rec):
    model = 'gpt-4o-mini'
    answer_llm = rag(rec, model=model)
    
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['text']

    return {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course'],
    }

In [ ]:
process_record_4o_mini(ground_truth[10])

In [ ]:
results_gpt4omini = []

In [ ]:
for record in tqdm(ground_truth):
    result = process_record_4o_mini(record)
    results_gpt4omini.append(result)

In [ ]:
df_gpt4o_mini = pd.DataFrame(results_gpt4omini)
df_gpt4o_mini.to_csv('data/results-gpt4o-mini.csv', index=False)

In [ ]:
similarity_4o_mini = []

for record in tqdm(results_gpt4omini):
    sim = compute_similarity(record)
    similarity_4o_mini.append(sim)

In [ ]:
df_gpt4o_mini['cosine'] = similarity_4o_mini
df_gpt4o_mini['cosine'].describe()

gpt4o 

```
count    1830.000000
mean        0.679129
std         0.217995
min        -0.153426
25%         0.591460
50%         0.734788
75%         0.835390
max         0.995339
Name: cosine, dtype: float64
```

In [ ]:
# sns.distplot(df_gpt35['cosine'], label='3.5')

sns.distplot(df_gpt4o['cosine'], label='4o')
sns.distplot(df_gpt4o_mini['cosine'], label='4o-mini')

plt.title("RAG LLM performance")
plt.xlabel("A->Q->A' Cosine Similarity")
plt.legend()

## LLM-as-a-Judge

In [ ]:
prompt1_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: {answer_orig}
Generated Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the original
answer and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [ ]:
df_sample = df_gpt4o_mini.sample(n=150, random_state=1)

In [ ]:
samples = df_sample.to_dict(orient='records')

In [ ]:
record = samples[0]
record

In [ ]:
prompt = prompt1_template.format(**record)
print(prompt)

In [ ]:
answer = llm(prompt, model='gpt-4o-mini')

In [ ]:
import json

In [ ]:
evaluations = []

for record in tqdm(samples):
    prompt = prompt1_template.format(**record)
    evaluation = llm(prompt, model='gpt-4o-mini')
    evaluations.append(evaluation)

In [ ]:
json_evaluations = []

for i, str_eval in enumerate(evaluations):
    json_eval = json.loads(str_eval)
    json_evaluations.append(json_eval)

In [ ]:
df_evaluations = pd.DataFrame(json_evaluations)

In [ ]:
df_evaluations.Relevance.value_counts()

In [ ]:
df_evaluations[df_evaluations.Relevance == 'NON_RELEVANT'] #.to_dict(orient='records')

In [ ]:
sample[4]

In [ ]:
prompt = prompt2_template.format(**record)
print(prompt)

In [ ]:
evaluation = llm(prompt, model='gpt-4o-mini')
print(evaluation)

In [ ]:
evaluations_2 = []

for record in tqdm(samples):
    prompt = prompt2_template.format(**record)
    evaluation = llm(prompt, model='gpt-4o-mini')
    evaluations_2.append(evaluation)

In [ ]:
json_evaluations_2 = []

for i, str_eval in enumerate(evaluations_2):
    json_eval = json.loads(str_eval)
    json_evaluations_2.append(json_eval)

In [ ]:
df_evaluations_2 = pd.DataFrame(json_evaluations_2)

In [ ]:
df_evaluations_2[df_evaluations_2.Relevance == 'NON_RELEVANT']

In [ ]:
samples[45]

## Saving all the data

In [ ]:
df_gpt4o.to_csv('data/results-gpt4o-cosine.csv', index=False)
df_gpt35.to_csv('data/results-gpt35-cosine.csv', index=False)
df_gpt4o_mini.to_csv('data/results-gpt4o-mini-cosine.csv', index=False)

In [ ]:
df_evaluations.to_csv('data/evaluations-aqa.csv', index=False)
df_evaluations_2.to_csv('data/evaluations-qa.csv', index=False)